In [1]:
import os
import torch
from tqdm.auto import tqdm
from diffusion_3d.chestct.diffusion.ldm.model import LDM3DLightning
from diffusion_3d.chestct.diffusion.ldm.config import get_config
from diffusion_3d.datasets.ct_rate import CTRATEDataModule
from arjcode.visualize import plot_scans

In [2]:
config = get_config()
config

Munch({'data': Munch({'csvpath': '/raid3/arjun/ct_pretraining/csvs/sources.csv', 'datapath': '/raid3/arjun/ct_pretraining/scans/', 'checkpointspath': '/raid3/arjun/checkpoints/diffusion_3d/', 'limited_dataset_size': None, 'allowed_spacings': ((0.4, 7), (-1, -1), (-1, -1)), 'allowed_shapes': ((96, -1), (256, -1), (256, -1)), 'train_augmentations': Munch({'_target_': 'monai.transforms.Compose', 'transforms': [Munch({'_target_': 'vision_architectures.transforms.croppad.CropForegroundWithCropTrackingd', 'keys': ['image'], 'source_key': 'image', 'allow_smaller': True}), Munch({'_target_': 'monai.transforms.ScaleIntensityRanged', 'keys': ['image'], 'a_min': -1000, 'a_max': 1000, 'b_min': 0.0, 'b_max': 1.0, 'clip': True}), Munch({'_target_': 'vision_architectures.transforms.croppad.RandSpatialCropSamplesWithCropTrackingd', 'keys': ['image'], 'roi_size': (76, 76, 76), 'max_roi_size': (115, 115, 115), 'random_size': True, 'num_samples': 10}), Munch({'_target_': 'vision_architectures.transforms.

In [3]:
checkpoint_path = r"/raid3/arjun/checkpoints/diffusion_3d/v4__2025_05_07/version_0/checkpoints/last.ckpt"
os.path.exists(checkpoint_path)

True

In [4]:
device = torch.device("cpu")
device = torch.device("cuda:1")

In [5]:
model = LDM3DLightning.load_from_checkpoint(checkpoint_path, map_location="cpu")
model = model.to(device)
model.eval()


LDM3DLightning(
  (vae): FrozenMaisiVAE(
    (model): AutoencoderKlMaisi(
      (encoder): MaisiEncoder(
        (blocks): ModuleList(
          (0): MaisiConvolution(
            (conv): Convolution(
              (conv): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            )
          )
          (1-2): 2 x MaisiResBlock(
            (norm1): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
            (conv1): MaisiConvolution(
              (conv): Convolution(
                (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
              )
            )
            (norm2): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
            (conv2): MaisiConvolution(
              (conv): Convolution(
                (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
              )
            )
            (nin_shortcut): Identity()
          )
          (3): MaisiDownsample(
           

# Check denoising

In [6]:
datamodule = CTRATEDataModule(config.data)

datamodule.setup("test")
dataloader = datamodule.test_dataloader()

test:   0%|          | 0/500 [00:00<?, ?it/s]

No. of test datapoints: 500


In [7]:
for batch in dataloader:
    break

batch = datamodule.transfer_batch_to_device(batch, device, 0)

In [8]:
image = batch["image"]
x0, spacings, offsets = model.process_batch(batch)

image.shape, x0.shape, spacings.shape, offsets.shape


(
    torch.Size([1, 1, 96, 96, 96]),
    torch.Size([1, 4, 24, 24, 24]),
    torch.Size([1, 3]),
    torch.Size([1, 3])
)

In [9]:
T = 750
noise = torch.randn_like(x0)
t = torch.tensor((T,), device=device)
guidance = torch.ones((2, 1), device=device)

In [10]:
xT = model.noise_scheduler.add_noise(x0, t, noise)

print(xT.min(), xT.max())
plot_scans([x0[0, 0].cpu(), xT[0, 0].cpu()], ["x0", f"x{T}"])

tensor(-4.7936, device='cuda:1') tensor(4.9009, device='cuda:1')


interactive(children=(IntSlider(value=0, description='z', max=23), Output()), _dom_classes=('widget-interact',…

In [ ]:
with torch.no_grad():
    xt = xT

    for timestep in tqdm([*range(T - 1, 0, -1)]):
        batch_size = xt.shape[0]

        noise_params = model.noise_scheduler.get_sqrt_one_minus_alpha_bars(t)

        noise_params_guidance = torch.ones((batch_size,)).to(x0)
        spacings_guidance = torch.ones((batch_size,)).to(x0)
        offsets_guidance = torch.ones((batch_size,)).to(x0)
        guidance = torch.stack([noise_params_guidance, spacings_guidance, offsets_guidance], dim=0).to(x0)

        noise_pred = model(xt, noise_params, spacings, offsets, guidance)

        x0_hat, xt_minus_1 = model.noise_scheduler.remove_noise(xt, t, noise_pred, eta=1.0)

        image_pred = model.vae.decode(x0_hat)

        if (timestep + 1) % 50 == 0 or t == 1:
            print(f"timestep: {timestep}")
            print(f"xt: {xt.min()}, {xt.max()}")
            print(f"noise: {noise.min()}, {noise.max()}")
            print(f"noise_pred: {noise_pred.min()}, {noise_pred.max()}")
            print(f"x0_hat: {x0_hat.min()}, {x0_hat.max()}")
            print(f"xt_minus_1: {xt_minus_1.min()}, {xt_minus_1.max()}")
            print(f"L2: {torch.nn.functional.mse_loss(noise_pred, noise)}")
            plot_scans(
                [
                    # noise_pred[0, 0, 10:11].cpu().numpy(),
                    # x0[0, 0, 10:11].detach().cpu().numpy(),
                    # x0_hat[0, 0, 10:11].detach().cpu().numpy(),
                    # xt_minus_1[0, 0, 10:11].detach().cpu().numpy(),
                    # xT[0, 0, 10:11].detach().cpu().numpy(),
                    image[0, 0, 10:11].cpu().numpy(),
                    image_pred[0, 0, 10:11].detach().cpu().numpy(),
                ],
                [
                    # "noise_pred",
                    # "x0",
                    # "x0_hat",
                    # "xt_minus_1",
                    # f"x{T}",
                    "image",
                    "image_pred",
                ],
            )

        xt = xt_minus_1

  0%|          | 0/749 [00:00<?, ?it/s]

timestep: 749
xt: -4.793623924255371, 4.900851726531982
noise: -4.89419412612915, 4.637295246124268
noise_pred: -4.393917083740234, 4.558681964874268
x0_hat: -1.9189693927764893, 2.4080562591552734
xt_minus_1: -4.751235485076904, 4.977629661560059
L2: 0.1265999972820282


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 699
xt: -4.0497822761535645, 4.577573299407959
noise: -4.89419412612915, 4.637295246124268
noise_pred: -3.809434175491333, 4.251965522766113
x0_hat: -1.6180058717727661, 2.2224161624908447
xt_minus_1: -4.1465020179748535, 4.467836856842041
L2: 0.43712544441223145


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

...

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 149
xt: -4.727411270141602, 3.935413360595703
noise: -4.89419412612915, 4.637295246124268
noise_pred: -4.1469879150390625, 4.027892589569092
x0_hat: -4.20801305770874, 3.777358293533325
xt_minus_1: -4.665591716766357, 4.007070541381836
L2: 1.7761648893356323


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 99
xt: -4.841803073883057, 4.510498046875
noise: -4.89419412612915, 4.637295246124268
noise_pred: -3.8044588565826416, 4.209552764892578
x0_hat: -5.0, 4.624021530151367
xt_minus_1: -4.921499729156494, 4.640794277191162
L2: 1.800473690032959


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

# Generate samples

In [36]:
from vision_architectures.schedulers.noise import CosineNoiseScheduler

In [46]:
num_samples = 1

spacings = torch.tensor([1.0, 0.5, 0.5], device=device)
guidance = torch.ones((2, num_samples), device=device)

In [47]:
xt = torch.randn(num_samples, config.model.in_channels, *config.input_size, device=device)
xt.shape

torch.Size([1, 1, 96, 96, 96])

In [ ]:
T = model.noise_scheduler.T
ddim_t_offset = T // 50
ddim_t_offset = 1

with torch.no_grad():
    for timestep in tqdm(range(T, 0, -ddim_t_offset)):
        t = torch.full((num_samples,), timestep, device=device)
        noise_pred = model(xt, t, spacings, guidance)

        x0_hat, xt_minus_1 = model.noise_scheduler.remove_noise(xt, t, noise_pred, prev_t_offset=ddim_t_offset, eta=1.0)

        if timestep % 50 == 0 or t == ddim_t_offset:
            print(f"timestep: {timestep}")
            print(f"xt: {xt.min()}, {xt.max()}")
            print(f"noise_pred: {noise_pred.min()}, {noise_pred.max()} | {noise_pred.mean()}, {noise_pred.std()}")
            print(f"x0_hat: {x0_hat.min()}, {x0_hat.max()}")
            print(f"xt_minus_1: {xt_minus_1.min()}, {xt_minus_1.max()}")
            plot_scans(
                [
                    noise_pred[0, 0, 40:41].cpu().numpy(),
                    x0_hat[0, 0, 40:41].detach().cpu().numpy(),
                    xt_minus_1[0, 0, 40:41].detach().cpu().numpy(),
                ],
                ["noise_pred", "x0_hat", f"xt_minus_{ddim_t_offset}"],
            )

        xt = xt_minus_1

  0%|          | 0/1000 [00:00<?, ?it/s]

timestep: 1000
xt: -5.207174777984619, 4.861217498779297
noise_pred: -5.242274761199951, 5.333522796630859 | 0.0021260040812194347, 0.9978124499320984
x0_hat: -5.0, 5.0
xt_minus_1: -4.888940334320068, 4.572839260101318


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 950
xt: -5.0, 5.0
noise_pred: -4.916719913482666, 6.680548191070557 | 0.001684906892478466, 1.0021768808364868
x0_hat: -5.0, 3.537106513977051
xt_minus_1: -4.857367515563965, 5.0


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

...

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 50
xt: -2.2994866371154785, 1.408508539199829
noise_pred: -4.12729024887085, 4.484928131103516 | -0.007741330657154322, 0.8706355094909668
x0_hat: -2.163160800933838, 1.3187366724014282
xt_minus_1: -2.3035640716552734, 1.3947064876556396


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

timestep: 1
xt: -2.211871385574341, 1.2753937244415283
noise_pred: -1.5889955759048462, 1.1262389421463013 | -0.033661700785160065, 0.18516898155212402
x0_hat: -2.211871385574341, 1.2753937244415283
xt_minus_1: -2.211871385574341, 1.2753937244415283


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…